# prepare_emissions_eas0b.ipynb

## Purpose
Prepare modified MAM3 emissions files for scenario **easb0**, as part of project [p17d-sulphur-eas-eqm](https://github.com/grandey/p17d-sulphur-eas-eqm).

## Requirements
- Standard CESM MAM3 emissions data files.
- Python modules mentioned in next cell.

## Author
Benjamin S. Grandey, 2017

In [1]:
import datetime
import netCDF4
import numpy as np
import os
import shutil
import stat

In [2]:
# Location of standard CESM MAM3 emissions data
mam3_dir = os.path.expandvars('$HOME/data/inputdataCESM/trop_mozart_aero/emis')

# Define bounds for region (East and Southeast Asia)
lon_bounds = (94, 161)
lat_bounds = (-10, 65)

# Species-level combinations that need modifying - those that contain sulphur emissions from energy/industry
species_level_list = ['so2_elev', 'so4_a1_elev', 'num_a1_elev',
                      'so4_a2_elev', 'num_a2_elev',
                      'so2_surf', 'so4_a1_surf', 'num_a1_surf',
                      'so4_a2_surf', 'num_a2_surf']

In [3]:
# Loop over species_level_list
for species_level in species_level_list:
    print(species_level)
    # Copy MAM3 emissions file
    in_filename = '{}/ar5_mam3_{}_2000_c090726.nc'.format(mam3_dir, species_level)
    out_filename = '{}_p17d_eas0b.nc'.format(species_level)
    if os.path.exists(out_filename):
        os.remove(out_filename)
    shutil.copy(in_filename, out_filename)
    os.chmod(out_filename, stat.S_IROTH | stat.S_IRUSR | stat.S_IWUSR)
    # Open file for editing
    out_nc = netCDF4.Dataset(out_filename, 'a')
    # Get longitude and latitude dimensions
    lon_x = out_nc['lon'][:]
    lat_y = out_nc['lat'][:]
    # Indices of region bounds.
    x_bot = np.where(lon_x >= lon_bounds[0])[0][0]
    x_top = np.where(lon_x <= lon_bounds[1])[0][-1]
    y_bot = np.where(lat_y >= lat_bounds[0])[0][0]
    y_top = np.where(lat_y <= lat_bounds[1])[0][-1]
    # Loop over variables to find anthropogenic sulphur categories
    for var_name in out_nc.variables:
        if (var_name[0:5] == 'emiss' or var_name[4:9] == 'emiss'):
            print('  Editing {}'.format(var_name))
            # Modify elevated data
            if species_level[-4:] == 'elev':
                # Load data
                nt, nz, ny, nx = out_nc[var_name][:].shape
                data_tzyx = out_nc[var_name][:]
                # Set to zero within region
                data_tzyx[:, :, y_bot:y_top+1, x_bot:x_top+1] = 0
                # Write modified data to file
                out_nc[var_name][:] = data_tzyx[:]
            # Modify surface data
            else:
                # Load data
                nt, ny, nx = out_nc[var_name][:].shape
                data_tyx = out_nc[var_name][:]
                # Set to zero within region
                data_tyx[:, y_bot:y_top+1, x_bot:x_top+1] = 0
                # Write modified data to file
                out_nc[var_name][:] = data_tyx[:] 
    # History attribute
    out_nc.history = (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')+
                       ': prepare_emissions_eas0b.ipynb: created '+out_filename+
                       ' using data from '+in_filename.split('/')[-1]+'.')
    out_nc.created_by = 'Benjamin S. Grandey, using prepare_emissions_eas0b.ipynb and ar5_mam3_* emissions files.'
    # Close file
    out_nc.close()
    print('  Written {}'.format(out_filename))

so2_elev
  Editing emiss_ene
  Editing emiss_ind
  Written so2_elev_p17d_eas0b.nc
so4_a1_elev
  Editing emiss_ene
  Editing emiss_ind
  Written so4_a1_elev_p17d_eas0b.nc
num_a1_elev
  Editing SO4_emiss_ene
  Editing SO4_emiss_ind
  Written num_a1_elev_p17d_eas0b.nc
so4_a2_elev
  Written so4_a2_elev_p17d_eas0b.nc
num_a2_elev
  Written num_a2_elev_p17d_eas0b.nc
so2_surf
  Editing emiss_awb
  Editing emiss_dom
  Editing emiss_tra
  Editing emiss_wst
  Editing emiss_shp
  Written so2_surf_p17d_eas0b.nc
so4_a1_surf
  Editing emiss_awb
  Editing emiss_wst
  Editing emiss_shp
  Written so4_a1_surf_p17d_eas0b.nc
num_a1_surf
  Editing SO4_emiss_awb
  Editing SO4_emiss_wst
  Editing SO4_emiss_shp
  Written num_a1_surf_p17d_eas0b.nc
so4_a2_surf
  Editing emiss_dom
  Editing emiss_tra
  Written so4_a2_surf_p17d_eas0b.nc
num_a2_surf
  Editing SO4_emiss_dom
  Editing SO4_emiss_tra
  Written num_a2_surf_p17d_eas0b.nc
